<a href="https://colab.research.google.com/github/ArkS0001/KAFKA-sample/blob/main/Kafka2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import queue
import threading
import time
import random

# Number of Producers, Consumers, and Brokers
NUM_PRODUCERS = 3
NUM_CONSUMERS = 5
NUM_BROKERS = 2

# Simulated Kafka Brokers (Queues)
global_broker_queues = [queue.Queue() for _ in range(NUM_BROKERS)]

# Consumer Groups (Each queue represents a group)
consumer_groups = {f"Group-{i+1}": queue.Queue() for i in range(NUM_CONSUMERS)}

# ✅ Complex Kafka Producer Simulation
def producer(producer_id):
    topics = ["Orders", "Payments", "Logs"]
    for i in range(10):  # Each producer sends 10 messages
        topic = random.choice(topics)
        message = f"Producer-{producer_id+1}: {topic} Event {i+1}"
        broker_id = i % NUM_BROKERS  # Distribute across brokers
        print(f"📤 Producer-{producer_id+1}: Sending -> {message} to Broker-{broker_id+1}")
        global_broker_queues[broker_id].put((topic, message))
        time.sleep(random.uniform(0.5, 1.5))  # Simulate random message intervals

    # Send STOP signal to all brokers
    for broker_queue in global_broker_queues:
        broker_queue.put(("STOP", None))

# ✅ Global Kafka Broker Simulation
def broker(broker_id):
    consumer_index = 0
    while True:
        topic, message = global_broker_queues[broker_id].get()
        if topic == "STOP":
            for cg_queue in consumer_groups.values():
                cg_queue.put(("STOP", None))
            break

        # Priority-based routing (Orders > Payments > Logs)
        group_id = f"Group-{consumer_index+1}"
        consumer_groups[group_id].put((topic, message))
        print(f"🔀 Broker-{broker_id+1}: Routed {message} to {group_id}")

        # Round-robin assignment
        consumer_index = (consumer_index + 1) % NUM_CONSUMERS

# ✅ Large-Scale Consumer Groups
def consumer(consumer_id, group_id):
    while True:
        topic, message = consumer_groups[group_id].get()
        if topic == "STOP":
            print(f"🔴 Consumer-{consumer_id+1} (Group {group_id}): Stopping.")
            break

        print(f"✅ Consumer-{consumer_id+1} (Group {group_id}): Processed {message}")
        time.sleep(random.uniform(1, 2))  # Simulate processing delay

# ✅ Run Multiple Producers, Brokers & Consumers
producer_threads = [threading.Thread(target=producer, args=(i,)) for i in range(NUM_PRODUCERS)]
broker_threads = [threading.Thread(target=broker, args=(i,)) for i in range(NUM_BROKERS)]
consumer_threads = [threading.Thread(target=consumer, args=(i, f"Group-{i+1}")) for i in range(NUM_CONSUMERS)]

# Start all threads
for pt in producer_threads:
    pt.start()
time.sleep(1)  # Allow producers to start

for bt in broker_threads:
    bt.start()
for ct in consumer_threads:
    ct.start()

# Wait for completion
for pt in producer_threads:
    pt.join()
for bt in broker_threads:
    bt.join()
for ct in consumer_threads:
    ct.join()

print("🚀 Enterprise Kafka-like pipeline simulation complete!")


📤 Producer-1: Sending -> Producer-1: Orders Event 1 to Broker-1
📤 Producer-2: Sending -> Producer-2: Orders Event 1 to Broker-1
📤 Producer-3: Sending -> Producer-3: Payments Event 1 to Broker-1
📤 Producer-1: Sending -> Producer-1: Payments Event 2 to Broker-2
📤 Producer-3: Sending -> Producer-3: Orders Event 2 to Broker-2
📤 Producer-2: Sending -> Producer-2: Logs Event 2 to Broker-2
🔀 Broker-1: Routed Producer-1: Orders Event 1 to Group-1
🔀 Broker-1: Routed Producer-2: Orders Event 1 to Group-2
🔀 Broker-1: Routed Producer-3: Payments Event 1 to Group-3
🔀 Broker-2: Routed Producer-1: Payments Event 2 to Group-1
🔀 Broker-2: Routed Producer-3: Orders Event 2 to Group-2
🔀 Broker-2: Routed Producer-2: Logs Event 2 to Group-3
✅ Consumer-1 (Group Group-1): Processed Producer-1: Orders Event 1
✅ Consumer-2 (Group Group-2): Processed Producer-2: Orders Event 1
✅ Consumer-3 (Group Group-3): Processed Producer-3: Payments Event 1
📤 Producer-3: Sending -> Producer-3: Payments Event 3 to Broker-1
🔀

In [5]:
import numpy as np
import random
import math
import queue
import threading
import time

# Number of Producers, Consumers, and Brokers
NUM_PRODUCERS = 3
NUM_CONSUMERS = 5
NUM_BROKERS = 2

# Simulated Kafka Brokers (Queues)
global_broker_queues = [queue.Queue() for _ in range(NUM_BROKERS)]

# Consumer Groups (Each queue represents a group)
consumer_groups = {f"Group-{i+1}": queue.Queue() for i in range(NUM_CONSUMERS)}

# Hyperparameters for qWhale
ALPHA = 0.1  # Learning rate
GAMMA = 0.9  # Discount factor
EPSILON = 0.1  # Exploration probability

# Initialize Q-table: State space (broker queues) x Actions (choose broker)
Q_table = np.zeros((NUM_BROKERS, NUM_BROKERS))

# qWhale Algorithm - Q-Learning Phase
def q_learning_update(state, action, reward, next_state):
    best_next_action = np.argmax(Q_table[next_state])
    Q_table[state][action] += ALPHA * (reward + GAMMA * Q_table[next_state][best_next_action] - Q_table[state][action])

# ✅ Whale Optimization Algorithm (WOA)
def whale_optimization_position_update(position, p, A, b, l, r):
    """Update position based on the WOA"""
    if p < 0.5:
        if abs(A) < 1:
            # Encircling Prey (Equation 2)
            return position + A * (b * np.random.rand() - position)
        else:
            # Searching for Prey (Equation 9)
            return position + A * (b * np.random.rand() - position)
    else:
        # Bubble Net Attack (Equation 6)
        return position + A * (position - b * np.random.rand())

# ✅ Select broker based on qWhale Algorithm
def qwhale_algorithm():
    """qWhale Load Balancing and Action Selection"""
    state = np.random.randint(0, NUM_BROKERS)
    action = np.random.randint(0, NUM_BROKERS)

    # Exploration or Exploitation based on epsilon-greedy
    if random.uniform(0, 1) < EPSILON:
        action = random.randint(0, NUM_BROKERS - 1)
    else:
        action = np.argmax(Q_table[state])

    return state, action

# Track broker usage to feed into qWhale
broker_usage_history = [0] * NUM_BROKERS

# ✅ Complex Kafka Producer Simulation
def producer(producer_id):
    topics = ["Orders", "Payments", "Logs"]
    for i in range(10):  # Each producer sends 10 messages
        topic = random.choice(topics)
        message = f"Producer-{producer_id+1}: {topic} Event {i+1}"

        state, action = qwhale_algorithm()  # Select broker based on qWhale
        broker_usage_history[action] += 1  # Update broker usage

        print(f"📤 Producer-{producer_id+1}: Sending -> {message} to Broker-{action+1}")
        global_broker_queues[action].put((topic, message))
        time.sleep(random.uniform(0.5, 1.5))  # Simulate random message intervals

    # Send STOP signal to all brokers
    for broker_queue in global_broker_queues:
        broker_queue.put(("STOP", None))

# ✅ Global Kafka Broker Simulation with qWhale Load Balancing
def broker(broker_id):
    consumer_index = 0
    while True:
        topic, message = global_broker_queues[broker_id].get()
        if topic == "STOP":
            for cg_queue in consumer_groups.values():
                cg_queue.put(("STOP", None))
            break

        # qWhale Routing for Consumer Selection
        group_id = f"Group-{qwhale_algorithm()[1] + 1}"
        consumer_groups[group_id].put((topic, message))
        print(f"🔀 Broker-{broker_id+1}: Routed {message} to {group_id}")

# ✅ Large-Scale Consumer Groups
def consumer(consumer_id, group_id):
    while True:
        topic, message = consumer_groups[group_id].get()
        if topic == "STOP":
            print(f"🔴 Consumer-{consumer_id+1} (Group {group_id}): Stopping.")
            break

        print(f"✅ Consumer-{consumer_id+1} (Group {group_id}): Processed {message}")
        time.sleep(random.uniform(1, 2))  # Simulate processing delay

# ✅ Run Multiple Producers, Brokers & Consumers
producer_threads = [threading.Thread(target=producer, args=(i,)) for i in range(NUM_PRODUCERS)]
broker_threads = [threading.Thread(target=broker, args=(i,)) for i in range(NUM_BROKERS)]
consumer_threads = [threading.Thread(target=consumer, args=(i, f"Group-{i+1}")) for i in range(NUM_CONSUMERS)]

# Start all threads
for pt in producer_threads:
    pt.start()
time.sleep(1)  # Allow producers to start

for bt in broker_threads:
    bt.start()
for ct in consumer_threads:
    ct.start()

# Wait for completion
for pt in producer_threads:
    pt.join()
for bt in broker_threads:
    bt.join()
for ct in consumer_threads:
    ct.join()

print("🚀 Enterprise Kafka-like pipeline simulation with qWhale load balancing complete!")


📤 Producer-1: Sending -> Producer-1: Payments Event 1 to Broker-2
📤 Producer-2: Sending -> Producer-2: Payments Event 1 to Broker-1
📤 Producer-3: Sending -> Producer-3: Logs Event 1 to Broker-1
📤 Producer-3: Sending -> Producer-3: Logs Event 2 to Broker-1
📤 Producer-2: Sending -> Producer-2: Logs Event 2 to Broker-1
🔀 Broker-1: Routed Producer-2: Payments Event 1 to Group-1
🔀 Broker-2: Routed Producer-1: Payments Event 1 to Group-1
✅ Consumer-1 (Group Group-1): Processed Producer-2: Payments Event 1
🔀 Broker-1: Routed Producer-3: Logs Event 1 to Group-1
🔀 Broker-1: Routed Producer-3: Logs Event 2 to Group-1
🔀 Broker-1: Routed Producer-2: Logs Event 2 to Group-2
✅ Consumer-2 (Group Group-2): Processed Producer-2: Logs Event 2
📤 Producer-1: Sending -> Producer-1: Logs Event 2 to Broker-1
🔀 Broker-1: Routed Producer-1: Logs Event 2 to Group-1
📤 Producer-3: Sending -> Producer-3: Logs Event 3 to Broker-1
🔀 Broker-1: Routed Producer-3: Logs Event 3 to Group-1
📤 Producer-2: Sending -> Produc

In [8]:
import numpy as np
import random
import math
import queue
import threading
import time
from concurrent.futures import ThreadPoolExecutor

# Number of Producers, Consumers, and Brokers
NUM_PRODUCERS = 3
NUM_CONSUMERS = 5
NUM_BROKERS = 2

# Simulated Kafka Brokers (Queues)
global_broker_queues = [queue.Queue() for _ in range(NUM_BROKERS)]

# Consumer Groups (Each queue represents a group)
consumer_groups = {f"Group-{i+1}": queue.Queue() for i in range(NUM_CONSUMERS)}

# Hyperparameters for qWhale
ALPHA = 0.1  # Learning rate
GAMMA = 0.9  # Discount factor
EPSILON = 0.1  # Exploration probability

# Initialize Q-table: State space (broker queues) x Actions (choose broker)
Q_table = np.zeros((NUM_BROKERS, NUM_BROKERS))

# qWhale Algorithm - Q-Learning Phase
def q_learning_update(state, action, reward, next_state):
    best_next_action = np.argmax(Q_table[next_state])
    Q_table[state][action] += ALPHA * (reward + GAMMA * Q_table[next_state][best_next_action] - Q_table[state][action])

# Whale Optimization Algorithm (WOA)
def whale_optimization_position_update(position, p, A, b, l, r):
    """Update position based on the WOA"""
    if p < 0.5:
        if abs(A) < 1:
            return position + A * (b * np.random.rand() - position)
        else:
            return position + A * (b * np.random.rand() - position)
    else:
        return position + A * (position - b * np.random.rand())

# Select broker based on qWhale Algorithm
def qwhale_algorithm():
    """qWhale Load Balancing and Action Selection"""
    state = np.random.randint(0, NUM_BROKERS)
    action = np.random.randint(0, NUM_BROKERS)

    if random.uniform(0, 1) < EPSILON:
        action = random.randint(0, NUM_BROKERS - 1)
    else:
        action = np.argmax(Q_table[state])

    return state, action

# Track broker usage to feed into qWhale
broker_usage_history = [0] * NUM_BROKERS

# Producer to Broker Load Balancing
def producer(producer_id):
    topics = ["Orders", "Payments", "Logs"]
    for i in range(10):
        topic = random.choice(topics)
        message = f"Producer-{producer_id+1}: {topic} Event {i+1}"

        state, action = qwhale_algorithm()
        broker_usage_history[action] += 1

        print(f"📤 Producer-{producer_id+1}: Sending -> {message} to Broker-{action+1}")
        global_broker_queues[action].put((topic, message))
        time.sleep(random.uniform(0.5, 1.5))

    for broker_queue in global_broker_queues:
        broker_queue.put(("STOP", None))

# Global Kafka Broker Simulation with qWhale Load Balancing
def broker(broker_id):
    while True:
        topic, message = global_broker_queues[broker_id].get()
        if topic == "STOP":
            break

        group_id = f"Group-{qwhale_algorithm()[1] + 1}"
        consumer_groups[group_id].put((topic, message))
        print(f"🔀 Broker-{broker_id+1}: Routed {message} to {group_id}")

# Large-Scale Consumer Groups
def consumer(consumer_id, group_id):
    while True:
        topic, message = consumer_groups[group_id].get()
        if topic == "STOP":
            print(f"🔴 Consumer-{consumer_id+1} (Group {group_id}): Stopping.")
            break

        print(f"✅ Consumer-{consumer_id+1} (Group {group_id}): Processed {message}")
        time.sleep(random.uniform(1, 2))

# Use ThreadPoolExecutor for scalable thread management
def run_system():
    with ThreadPoolExecutor(max_workers=NUM_PRODUCERS + NUM_CONSUMERS + NUM_BROKERS) as executor:
        producer_threads = [executor.submit(producer, i) for i in range(NUM_PRODUCERS)]
        broker_threads = [executor.submit(broker, i) for i in range(NUM_BROKERS)]
        consumer_threads = [executor.submit(consumer, i, f"Group-{i+1}") for i in range(NUM_CONSUMERS)]

        # Wait for all threads to complete
        for t in producer_threads + broker_threads + consumer_threads:
            t.result()

    print("🚀 Enterprise Kafka-like pipeline simulation with qWhale load balancing complete!")

# Start the system
run_system()


📤 Producer-1: Sending -> Producer-1: Logs Event 1 to Broker-1📤 Producer-2: Sending -> Producer-2: Payments Event 1 to Broker-1
📤 Producer-3: Sending -> Producer-3: Payments Event 1 to Broker-1

🔀 Broker-1: Routed Producer-3: Payments Event 1 to Group-1
🔀 Broker-1: Routed Producer-2: Payments Event 1 to Group-1
🔀 Broker-1: Routed Producer-1: Logs Event 1 to Group-1
✅ Consumer-1 (Group Group-1): Processed Producer-3: Payments Event 1
📤 Producer-2: Sending -> Producer-2: Payments Event 2 to Broker-1
🔀 Broker-1: Routed Producer-2: Payments Event 2 to Group-1
📤 Producer-3: Sending -> Producer-3: Orders Event 2 to Broker-1
🔀 Broker-1: Routed Producer-3: Orders Event 2 to Group-1
📤 Producer-1: Sending -> Producer-1: Logs Event 2 to Broker-1
🔀 Broker-1: Routed Producer-1: Logs Event 2 to Group-1
✅ Consumer-1 (Group Group-1): Processed Producer-2: Payments Event 1
📤 Producer-3: Sending -> Producer-3: Logs Event 3 to Broker-1
🔀 Broker-1: Routed Producer-3: Logs Event 3 to Group-1
📤 Producer-1: S

KeyboardInterrupt: 

In [6]:
import time
import threading
import queue
import random
import numpy as np

# Parameters
NUM_PRODUCERS = 3
NUM_CONSUMERS = 5
NUM_BROKERS = 2

# Metrics
broker_load_round_robin = [0] * NUM_BROKERS
broker_load_qwhale = [0] * NUM_BROKERS
message_count_round_robin = 0
message_count_qwhale = 0

# Kafka-like broker queues
global_broker_queues = [queue.Queue() for _ in range(NUM_BROKERS)]
consumer_groups = {f"Group-{i+1}": queue.Queue() for i in range(NUM_CONSUMERS)}

# qWhale parameters
ALPHA = 0.1  # Learning rate
GAMMA = 0.9  # Discount factor
EPSILON = 0.1  # Exploration probability
Q_table = np.zeros((NUM_BROKERS, NUM_BROKERS))

# Function to update Q-table (Q-learning)
def q_learning_update(state, action, reward, next_state):
    best_next_action = np.argmax(Q_table[next_state])
    Q_table[state][action] += ALPHA * (reward + GAMMA * Q_table[next_state][best_next_action] - Q_table[state][action])

# qWhale-based broker selection
def qwhale_algorithm():
    state = np.random.randint(0, NUM_BROKERS)
    action = np.random.randint(0, NUM_BROKERS)

    # Exploration or exploitation
    if random.uniform(0, 1) < EPSILON:
        action = random.randint(0, NUM_BROKERS - 1)
    else:
        action = np.argmax(Q_table[state])
    return state, action

# Round Robin broker selection
def round_robin_broker():
    return random.randint(0, NUM_BROKERS - 1)

# Producer that sends messages to brokers
def producer(producer_id, selection_method):
    global message_count_round_robin, message_count_qwhale

    topics = ["Orders", "Payments", "Logs"]
    for i in range(10):  # Each producer sends 10 messages
        topic = random.choice(topics)
        message = f"Producer-{producer_id+1}: {topic} Event {i+1}"

        if selection_method == 'qwhale':
            state, action = qwhale_algorithm()
            broker_load_qwhale[action] += 1  # Update broker load for qWhale
            message_count_qwhale += 1
        elif selection_method == 'round_robin':
            action = round_robin_broker()
            broker_load_round_robin[action] += 1  # Update broker load for Round Robin
            message_count_round_robin += 1

        print(f"📤 Producer-{producer_id+1}: Sending -> {message} to Broker-{action+1}")
        global_broker_queues[action].put((topic, message))
        time.sleep(random.uniform(0.5, 1.5))  # Simulate random message intervals

# Broker that routes messages to consumers
def broker(broker_id):
    while True:
        topic, message = global_broker_queues[broker_id].get()
        if topic == "STOP":
            break

        # Simulate consumer selection based on qWhale routing (or round robin)
        group_id = f"Group-{random.randint(1, NUM_CONSUMERS)}"
        consumer_groups[group_id].put((topic, message))
        print(f"🔀 Broker-{broker_id+1}: Routed {message} to {group_id}")

# Consumer that processes messages
def consumer(consumer_id, group_id):
    while True:
        topic, message = consumer_groups[group_id].get()
        if topic == "STOP":
            break
        print(f"✅ Consumer-{consumer_id+1} (Group {group_id}): Processed {message}")
        time.sleep(random.uniform(1, 2))  # Simulate processing delay

# Function to start simulation
def start_simulation(selection_method):
    global broker_load_round_robin, broker_load_qwhale, message_count_round_robin, message_count_qwhale

    # Reset metrics
    broker_load_round_robin = [0] * NUM_BROKERS
    broker_load_qwhale = [0] * NUM_BROKERS
    message_count_round_robin = 0
    message_count_qwhale = 0

    # Start producer threads
    producer_threads = [threading.Thread(target=producer, args=(i, selection_method)) for i in range(NUM_PRODUCERS)]
    for pt in producer_threads:
        pt.start()
    time.sleep(1)  # Allow producers to start

    # Start broker threads
    broker_threads = [threading.Thread(target=broker, args=(i,)) for i in range(NUM_BROKERS)]
    for bt in broker_threads:
        bt.start()

    # Start consumer threads
    consumer_threads = [threading.Thread(target=consumer, args=(i, f"Group-{i+1}")) for i in range(NUM_CONSUMERS)]
    for ct in consumer_threads:
        ct.start()

    # Wait for completion
    for pt in producer_threads:
        pt.join()
    for bt in broker_threads:
        bt.join()
    for ct in consumer_threads:
        ct.join()

    # Results
    if selection_method == 'qwhale':
        print("\n=== qWhale Load Balancing Results ===")
        print("Broker Load (qWhale):", broker_load_qwhale)
        print("Messages Sent (qWhale):", message_count_qwhale)
    else:
        print("\n=== Round Robin Load Balancing Results ===")
        print("Broker Load (Round Robin):", broker_load_round_robin)
        print("Messages Sent (Round Robin):", message_count_round_robin)

    print(f"Messages Processed: {message_count_round_robin if selection_method == 'round_robin' else message_count_qwhale}")

# Run and compare both methods
print("Starting simulation with Round Robin scheduling...")
start_simulation('round_robin')
time.sleep(2)

print("\nStarting simulation with qWhale scheduling...")
start_simulation('qwhale')


Starting simulation with Round Robin scheduling...
📤 Producer-1: Sending -> Producer-1: Logs Event 1 to Broker-2📤 Producer-2: Sending -> Producer-2: Payments Event 1 to Broker-2

📤 Producer-3: Sending -> Producer-3: Payments Event 1 to Broker-1
📤 Producer-3: Sending -> Producer-3: Logs Event 2 to Broker-1
🔀 Broker-1: Routed Producer-3: Payments Event 1 to Group-2🔀 Broker-2: Routed Producer-2: Payments Event 1 to Group-3
🔀 Broker-2: Routed Producer-1: Logs Event 1 to Group-1

✅ Consumer-1 (Group Group-1): Processed Producer-1: Logs Event 1
✅ Consumer-2 (Group Group-2): Processed Producer-3: Payments Event 1
🔀 Broker-1: Routed Producer-3: Logs Event 2 to Group-5✅ Consumer-3 (Group Group-3): Processed Producer-2: Payments Event 1

✅ Consumer-5 (Group Group-5): Processed Producer-3: Logs Event 2
📤 Producer-2: Sending -> Producer-2: Payments Event 2 to Broker-2
🔀 Broker-2: Routed Producer-2: Payments Event 2 to Group-2
📤 Producer-1: Sending -> Producer-1: Orders Event 2 to Broker-1
🔀 Broker

KeyboardInterrupt: 

In [7]:
import time
import threading
import queue
import random
import numpy as np

# Parameters
NUM_PRODUCERS = 3
NUM_CONSUMERS = 5
NUM_BROKERS = 2
MAX_MESSAGES = 10  # Number of messages each producer sends

# Metrics
broker_load_round_robin = [0] * NUM_BROKERS
broker_load_qwhale = [0] * NUM_BROKERS
message_count_round_robin = 0
message_count_qwhale = 0

# Kafka-like broker queues
global_broker_queues = [queue.Queue() for _ in range(NUM_BROKERS)]
consumer_groups = {f"Group-{i+1}": queue.Queue() for i in range(NUM_CONSUMERS)}

# qWhale parameters
ALPHA = 0.1  # Learning rate
GAMMA = 0.9  # Discount factor
EPSILON = 0.1  # Exploration probability
Q_table = np.zeros((NUM_BROKERS, NUM_BROKERS))

# Function to update Q-table (Q-learning)
def q_learning_update(state, action, reward, next_state):
    best_next_action = np.argmax(Q_table[next_state])
    Q_table[state][action] += ALPHA * (reward + GAMMA * Q_table[next_state][best_next_action] - Q_table[state][action])

# qWhale-based broker selection
def qwhale_algorithm():
    state = np.random.randint(0, NUM_BROKERS)
    action = np.random.randint(0, NUM_BROKERS)

    # Exploration or exploitation
    if random.uniform(0, 1) < EPSILON:
        action = random.randint(0, NUM_BROKERS - 1)
    else:
        action = np.argmax(Q_table[state])
    return state, action

# Round Robin broker selection
def round_robin_broker():
    return random.randint(0, NUM_BROKERS - 1)

# Producer that sends messages to brokers
def producer(producer_id, selection_method, stop_event):
    global message_count_round_robin, message_count_qwhale

    topics = ["Orders", "Payments", "Logs"]
    for i in range(MAX_MESSAGES):  # Each producer sends MAX_MESSAGES messages
        topic = random.choice(topics)
        message = f"Producer-{producer_id+1}: {topic} Event {i+1}"

        if selection_method == 'qwhale':
            state, action = qwhale_algorithm()
            broker_load_qwhale[action] += 1  # Update broker load for qWhale
            message_count_qwhale += 1
        elif selection_method == 'round_robin':
            action = round_robin_broker()
            broker_load_round_robin[action] += 1  # Update broker load for Round Robin
            message_count_round_robin += 1

        print(f"📤 Producer-{producer_id+1}: Sending -> {message} to Broker-{action+1}")
        global_broker_queues[action].put((topic, message))
        time.sleep(random.uniform(0.5, 1.5))  # Simulate random message intervals

    # Signal broker to stop after all messages are produced
    stop_event.set()

# Broker that routes messages to consumers
def broker(broker_id, stop_event):
    while not stop_event.is_set() or any(not q.empty() for q in global_broker_queues):
        topic, message = global_broker_queues[broker_id].get()
        if topic == "STOP":
            break

        # Simulate consumer selection based on qWhale routing (or round robin)
        group_id = f"Group-{random.randint(1, NUM_CONSUMERS)}"
        consumer_groups[group_id].put((topic, message))
        print(f"🔀 Broker-{broker_id+1}: Routed {message} to {group_id}")
        time.sleep(random.uniform(0.5, 1.5))  # Simulate some processing time

# Consumer that processes messages
def consumer(consumer_id, group_id, stop_event):
    while not stop_event.is_set() or any(not q.empty() for q in consumer_groups.values()):
        topic, message = consumer_groups[group_id].get()
        if topic == "STOP":
            break
        print(f"✅ Consumer-{consumer_id+1} (Group {group_id}): Processed {message}")
        time.sleep(random.uniform(1, 2))  # Simulate processing delay

# Function to start simulation
def start_simulation(selection_method):
    global broker_load_round_robin, broker_load_qwhale, message_count_round_robin, message_count_qwhale

    # Reset metrics
    broker_load_round_robin = [0] * NUM_BROKERS
    broker_load_qwhale = [0] * NUM_BROKERS
    message_count_round_robin = 0
    message_count_qwhale = 0

    # Create stop events
    stop_event_producer = threading.Event()
    stop_event_broker = threading.Event()
    stop_event_consumer = threading.Event()

    # Start producer threads
    producer_threads = [threading.Thread(target=producer, args=(i, selection_method, stop_event_producer)) for i in range(NUM_PRODUCERS)]
    for pt in producer_threads:
        pt.start()

    # Start broker threads
    broker_threads = [threading.Thread(target=broker, args=(i, stop_event_broker)) for i in range(NUM_BROKERS)]
    for bt in broker_threads:
        bt.start()

    # Start consumer threads
    consumer_threads = [threading.Thread(target=consumer, args=(i, f"Group-{i+1}", stop_event_consumer)) for i in range(NUM_CONSUMERS)]
    for ct in consumer_threads:
        ct.start()

    # Wait for producers to finish
    for pt in producer_threads:
        pt.join()

    # Signal brokers and consumers to stop
    stop_event_broker.set()
    stop_event_consumer.set()

    # Wait for broker and consumer threads to finish
    for bt in broker_threads:
        bt.join()
    for ct in consumer_threads:
        ct.join()

    # Results
    if selection_method == 'qwhale':
        print("\n=== qWhale Load Balancing Results ===")
        print("Broker Load (qWhale):", broker_load_qwhale)
        print("Messages Sent (qWhale):", message_count_qwhale)
    else:
        print("\n=== Round Robin Load Balancing Results ===")
        print("Broker Load (Round Robin):", broker_load_round_robin)
        print("Messages Sent (Round Robin):", message_count_round_robin)

    print(f"Messages Processed: {message_count_round_robin if selection_method == 'round_robin' else message_count_qwhale}")

# Run and compare both methods
print("Starting simulation with Round Robin scheduling...")
start_simulation('round_robin')
time.sleep(2)

print("\nStarting simulation with qWhale scheduling...")
start_simulation('qwhale')

Starting simulation with Round Robin scheduling...
📤 Producer-1: Sending -> Producer-1: Logs Event 1 to Broker-2
📤 Producer-2: Sending -> Producer-2: Orders Event 1 to Broker-1
📤 Producer-3: Sending -> Producer-3: Orders Event 1 to Broker-1
🔀 Broker-1: Routed Producer-2: Orders Event 1 to Group-4🔀 Broker-2: Routed Producer-1: Logs Event 1 to Group-3

✅ Consumer-3 (Group Group-3): Processed Producer-1: Logs Event 1
✅ Consumer-4 (Group Group-4): Processed Producer-2: Orders Event 1
📤 Producer-3: Sending -> Producer-3: Orders Event 2 to Broker-2
📤 Producer-2: Sending -> Producer-2: Logs Event 2 to Broker-1
🔀 Broker-1: Routed Producer-3: Orders Event 1 to Group-3
✅ Consumer-3 (Group Group-3): Processed Producer-3: Orders Event 1
📤 Producer-1: Sending -> Producer-1: Logs Event 2 to Broker-2
🔀 Broker-2: Routed Producer-3: Orders Event 2 to Group-5✅ Consumer-5 (Group Group-5): Processed Producer-3: Orders Event 2

📤 Producer-3: Sending -> Producer-3: Logs Event 3 to Broker-1
🔀 Broker-1: Route

KeyboardInterrupt: 